In [2]:
import arcpy
import cx_Oracle
import pandas as pd

In [3]:
def connect_to_DB (username,password,hostname):
    """ Returns a connection to Oracle database"""
    try:
        connection = cx_Oracle.connect(username, password, hostname, encoding="UTF-8")
        print  ("Successffuly connected to the database")
    except:
        raise Exception('Connection failed! Please verifiy your login parameters')

    return connection

In [ ]:
# connect to BCGW
hostname = 'bcgw.bcgov/idwprod1.bcgov'
username = input("Enter your BCGW username:")
password = input("Enter your BCGW password:")

In [21]:
connection = connect_to_DB (username,password,hostname)

Successffuly connected to the database


In [22]:
def get_wkt_srid (fc):
    """Returns the SRID and WKT string of a shapefile or Feature Class"""
    # get SRID
    sr = arcpy.Describe(fc).spatialReference
    srid = sr.factorycode

    # get WKT string
    cursor = arcpy.da.SearchCursor(fc, ["SHAPE@WKT"])
    for row in cursor:
        wkt = row[0]
        break # use only first geometry. just for demonstration.

    del row
    del cursor

    return wkt, srid

In [23]:
# input AOI: shp or feature class.
fc = r"\\path_to_file\file.shp"

In [24]:
# get the WKT string and SRID of the shp
wkt, srid = get_wkt_srid (fc)
print (srid)
print (wkt)

3005
POINT (1150040.4250008576 463057.94102919661)


In [27]:
# just an exemple of query: get all Log Handling Tenures within 10 km from point location (Nanaimo).
# this can be replaced with Allan's SQL code that shows both historic and active tenures.
query = """
            SELECT*
            FROM WHSE_TANTALIS.TA_CROWN_TENURES_SVW t
            WHERE t.TENURE_SUBPURPOSE = 'LOG HANDLING/STORAGE'
            AND t.TENURE_STAGE = 'TENURE'
            AND SDO_WITHIN_DISTANCE(t.SHAPE,
                                    SDO_GEOMETRY('{}', {}),
                                    'distance = 10000') = 'TRUE'
        """.format(wkt,srid)

In [28]:
#run query and print results
df = pd.read_sql(query, con=connection)
df

,INTRID_SID,TENURE_STAGE,TENURE_STATUS,TENURE_TYPE,TENURE_SUBTYPE,TENURE_PURPOSE,TENURE_SUBPURPOSE,CROWN_LANDS_FILE,APPLICATION_TYPE_CDE,TENURE_DOCUMENT,TENURE_EXPIRY,TENURE_LOCATION,TENURE_LEGAL_DESCRIPTION,TENURE_AREA_DERIVATION,TENURE_AREA_IN_HECTARES,RESPONSIBLE_BUSINESS_UNIT,DISPOSITION_TRANSACTION_SID,CODE_CHR_STAGE,FEATURE_CODE,SHAPE,FEATURE_AREA_SQM,FEATURE_LENGTH_M,OBJECTID,SE_ANNO_CAD_DATA
0,953830,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,0175352,REP,V927015,2027-06-30,GABRIOLA IS.,"District Lot 219, Nanaimo District.",From Survey,11.160,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,927015,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x793c...,87094.5069,2588.3109,51784,None
1,928439,TENURE,DISPOSITION IN GOOD STANDING,LICENCE,LICENCE OF OCCUPATION,INDUSTRIAL,LOG HANDLING/STORAGE,1412516,NEW,111497,2024-11-30,DUKE POINT,Uunsurveyed Crown foreshore or land covered by...,Calculated automatically,1.861,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,840960,T,FL98000500,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x793c...,18610.0233,639.1207,39508,None
2,968193,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,0175350,REP,V937231,2041-02-01,Northumberland Channel,"Block A of District Lot 215, Nanaimo District,...",Estimated,23.130,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,937231,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,232341.7444,4084.4169,61106,None
3,935009,TENURE,DISPOSITION IN GOOD STANDING,LICENCE,LICENCE OF OCCUPATION,INDUSTRIAL,LOG HANDLING/STORAGE,1408363,REP,V911873,2025-01-15,GABRIOLA BLUFFS,Unsurveyed Crown foreshore or land covered by ...,Calculated automatically,3.311,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,911873,T,FL98000500,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,33114.2653,1198.3108,42716,None
4,953834,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,0286558,REP,V927019,2027-06-30,GABRIOLA ISLAND,"District Lot 86, Nanaimo District",From Survey,2.680,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,927019,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,22355.5162,807.8779,51788,None
5,920265,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,0314351,REP,V899725,2043-05-31,GABRIOLA ISLAND,"DISTRICT LOT 236, NANAIMO DISTRICT",From Survey,6.961,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,899725,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,54775.0933,1810.4110,35382,None
6,954000,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,1400767,REP,V927181,2041-01-25,DUKE POINT,"District Lot 475, Nanaimo District",From Survey,17.410,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,927181,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,172390.6856,1655.4176,514,None
7,135716,TENURE,DISPOSITION IN GOOD STANDING,LEASE,STANDARD LEASE,INDUSTRIAL,LOG HANDLING/STORAGE,0175753,REP,104096,2022-04-01,HARMAC,"DISTRICT LOT 216, NANAIMO DISTRICT; DISTRICT L...",From Survey,38.940,VI - LAND MGMNT - VANCOUVER ISLAND SERVICE REGION,142540,T,FL98000400,<cx_Oracle.Object MDSYS.SDO_GEOMETRY at 0x796e...,405551.4490,5277.0925,746,None
